In [224]:
from selenium import webdriver
from bs4  import BeautifulSoup
import time
import csv
import os
import shutil
import codecs
import pandas as pd
import numpy as np
import random

In [261]:
data = r'C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)'
file_paths = []
#loop to get append file_paths 
for root, directories, files in os.walk(data):
    for filename in files:
        if ".html" in filename:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            #to append
            file_paths.append(filepath)

In [262]:
messages = []
for file in file_paths:    
    soup = BeautifulSoup(codecs.open(file, 'r', "utf-8").read())
    messages.extend(soup.findAll('div', attrs={'class':'message default clearfix'}))
    messages.extend(soup.findAll('div', attrs={'class':'message default clearfix joined'}))
    
print(len(contents))

4500


In [263]:
current_ids, previous_ids, message_texts,message_statuses = [],[],[],[]

for meg in messages:
    current_id = meg.get("id")[7:]
    current_id = int(current_id)
    
    try:
        previous_id = meg.find("div", {"class":"reply_to details"}).a.get("href")[14:]
        previous_id = int(previous_id)
    except:
        previous_id = 0
    
    try:
        message_text = meg.find("div", {"class":"description"})
        message_text = message_text.text.strip()
    except:
        try:
            message_text = meg.find("div", {"class":"text"})
            message_text = message_text.text.strip()
        except:
            continue
    
    try:
        message_status = meg.find("div", {"class":"status details"})
        message_status = message_status.text.strip().split(",")[0]
    except:
        message_status = None
    
    if "Not included, change data exporting settings to download." not in message_text and "/" not in message_text and "Try command /" not in message_text:
        current_ids.append(current_id)
        previous_ids.append(previous_id)
        message_texts.append(message_text)
        message_statuses.append(message_status)

In [264]:
dataset = pd.DataFrame({
    "current_id":current_ids,
    "previous_id":previous_ids,
    "message_text":message_texts,
    "message_status":message_statuses
    })

In [265]:
np.sum(dataset.isnull())

current_id             0
previous_id            0
message_text           0
message_status    413818
dtype: int64

In [266]:
def readPreviousMain(currentID):
    pm = dataset[dataset["previous_id"] == currentID].iloc[:,2].values
    pm = str(pm)

    cm = dataset[dataset["current_id"] == currentID].iloc[:,2].values
    cm = str(cm)

    print("Previous Message: ", pm)
    print("Current Message: ", cm)

In [267]:
readPreviousMain(5500021)

Previous Message:  []
Current Message:  ['I m also free You got free from ?']


In [303]:
readPreviousMain(5500036)

Previous Message:  []
Current Message:  ['Hi']


In [304]:
readPreviousMain(5500023)

Previous Message:  ['Chhii Omg']
Current Message:  ['Vo to funny hai sirf 😂🙈👀']


In [270]:
dataset.to_csv(r'C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\dataset.csv')

,current_id,previous_id,message_text,message_status
0,5500016,5491000,😂,None
1,5500021,5500000,I m also free You got free from ?,None
2,5500023,5500006,Vo to funny hai sirf 😂🙈👀,None
3,5500031,5494639,शुक्रिया,None
4,5500036,5499996,Hi,None
...,...,...,...,...
413828,5688350,0,Wbu?,None
413829,5688385,0,Koi hamse bhi baate Kar lo,None
413830,5688388,0,😂😂,None
413831,5688527,0,??,None


In [271]:
dataset.to_csv(r'C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\dataset.csv')

In [92]:
dataset = pd.read_csv(r"C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\dataset.csv")
dataset = dataset.iloc[:,1:]

In [93]:
messages_set = list(set(dataset.message_text))
exclude_text = ["bet", "Level", "YOUR PROFILE Messages sent here","LEADERBOARD","Local","City | Informations","Статистика пользователя" , "Report", "coinsOh no!","Status | Informations of" ,"coins", "\\u"]

final_messages_set = []
for i in messages_set:
    valid = True
    for j in exclude_text:
        if j in i:
            valid = False
    if valid:
        final_messages_set.append(i)
        
#removing only usernames
final_messages_set = [i for i in final_messages_set if i[0] != "@" and len(i.split(" ")) != 1]        
len(final_messages_set)

159072

In [197]:
def readPreviousMain(currentID):
    question = dataset[dataset["previous_id"] == currentID]
    question_str = str(question.message_text.values[0])
    if question.message_status.isna().bool() == False:
        question_str +=  " " + str(question.message_status.values[0])

    ans = dataset[dataset["current_id"] == currentID]
    ans_str = str(ans.message_text.values[0])
    if ans.message_status.isna().bool() == False:
        ans_str += " " + str(ans.message_status.values[0])
    
    #print("Previous Message: ", question_str)
    #print("Current Message: ", ans_str)
    return question_str, ans_str

In [215]:
question_message_text = []
ans_message_text = []
for message in final_messages_set:
    temp_dataset = dataset[dataset["message_text"] == message]
    try:
        q, a = readPreviousMain(int(temp_dataset.current_id.values[0]))
        question_message_text.append(q)
        ans_message_text.append(a)
    except:
        pass

In [216]:
output = pd.DataFrame({
    "question_message_text": question_message_text,
    "ans_message_text":ans_message_text
})
output.to_csv(r"C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\question_and_answer.csv")

In [217]:
len(question_message_text)

67665

DilogFlow

In [ ]:
def create_intent(project_id, display_name, training_phrases_parts, message_texts):
    """Create an intent of the given intent type."""
    from google.cloud import dialogflow

    intents_client = dialogflow.IntentsClient()

    parent = dialogflow.AgentsClient.agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part = dialogflow.Intent.TrainingPhrase.Part(text=training_phrases_part)
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.Intent.TrainingPhrase(parts=[part])
        training_phrases.append(training_phrase)

    text = dialogflow.Intent.Message.Text(text=message_texts)
    message = dialogflow.Intent.Message(text=text)

    intent = dialogflow.Intent(
        display_name=display_name, training_phrases=training_phrases, messages=[message]
    )

    response = intents_client.create_intent(
        request={"parent": parent, "intent": intent}
    )

    print("Intent created: {}".format(response))
    

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hindienglishchatbot-3d57939ede44.json"

In [219]:
question_ans = pd.read_csv(r"C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\question_and_answer.csv")
question_ans = question_ans.iloc[:,1:]
question_ans

,question_message_text,ans_message_text
0,Jammu,Kaha rhte hai not rhta hoo
1,😂😂,report kar diye h don't worry
2,Years...👀y,It will take years 😂
3,Mat kr sab aisi hei hai,"𝘉𝘰𝘭𝘢 𝘯𝘢, 𝘮𝘶𝘫𝘩𝘦 𝘬𝘪𝘴𝘪 𝘴𝘦 𝘣𝘩𝘪 𝘱𝘳𝘴𝘯𝘭 𝘱𝘦𝘳 𝘣𝘢𝘢𝘵 𝘯𝘩𝘪 ..."
4,Okay bhai bol dungi😂,बहन का ....भाई है
...,...,...
67660,kkk dude,when mute put expiry date and reason plz
67661,What should I break,Do it break it quick
67662,Thanks🙂,😳 wow
67663,😇,OK Np


In [226]:
training_phrases_parts = ["who is developer of this?", "who has developed it", "who is admin name", "who is admin of this"]
message_texts = ["Preyash Patel is our developer, here you can talk with him https://www.linkedin.com/in/preyash2047/"]
create_intent(
    project_id="hindienglishchatbot",
    display_name="about",
    training_phrases_parts = training_phrases_parts,
    message_texts = message_texts
)

Intent created: name: "projects/hindienglishchatbot/agent/intents/187689ea-729c-4422-aa02-d28c6a8ddd62"
display_name: "about"
priority: 500000
messages {
  text {
    text: "Preyash Patel is our developer, here you can talk with him https://www.linkedin.com/in/preyash2047/"
  }
}



InvalidArgument: 400 com.google.apps.framework.request.BadRequestException: Intent with the display name 'about' already exists.

In [ ]:
for i in range(len(question_ans)):    
    training_phrases_parts = question_ans.iloc[i,0]
    message_texts = question_ans.iloc[i,1]
    create_intent(
        project_id="hindienglishchatbot",
        display_name="intent_" + str(i),
        training_phrases_parts = training_phrases_parts,
        message_texts = message_texts
    )
    time.sleep(random.randint(5,10))

Intent created: name: "projects/hindienglishchatbot/agent/intents/7e52420a-7927-47bf-bbb4-8e6793ff0e86"
display_name: "intent_0"
priority: 500000
messages {
  text {
    text: "K"
    text: "a"
    text: "h"
    text: "a"
    text: " "
    text: "r"
    text: "h"
    text: "t"
    text: "e"
    text: " "
    text: "h"
    text: "a"
    text: "i"
    text: " "
    text: "n"
    text: "o"
    text: "t"
    text: " "
    text: "r"
    text: "h"
    text: "t"
    text: "a"
    text: " "
    text: "h"
    text: "o"
    text: "o"
  }
}

Intent created: name: "projects/hindienglishchatbot/agent/intents/a12a89a8-b88a-4735-98e2-b3edeafba53e"
display_name: "intent_1"
priority: 500000
messages {
  text {
    text: "r"
    text: "e"
    text: "p"
    text: "o"
    text: "r"
    text: "t"
    text: " "
    text: "k"
    text: "a"
    text: "r"
    text: " "
    text: "d"
    text: "i"
    text: "y"
    text: "e"
    text: " "
    text: "h"
    text: " "
    text: "d"
    text: "o"
    text: "n"
   